In [1]:
import os
import subprocess
from Bio import SeqIO
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
import glob
from Bio import SeqRecord
import pandas as pd
#pip install genomicranges #this worked 
import genomicranges as gr
from Bio.Seq import Seq
#from Bio.SeqRecord import SeqRecord 
import pyranges as pr
from Bio import pairwise2
from Bio.SeqUtils import GC

In [342]:
def make_sbatch_file(filename): 
    
    my_list = ["#!/bin/bash", 
               "#SBATCH --job-name=all_bz", 
               "#SBATCH --nodes=1",  
               "#SBATCH --ntasks=1",                     
               "#SBATCH --cpus-per-task=10",              
               "#SBATCH --mem=20gb",                    
               "#SBATCH --partition=20",                
               "##SBATCH --output all_bz-%j.out",  
               "#SBATCH --mail-type=ALL",               
               "#SBATCH --mail-user=hlharris@wi.mit.edu"] 
    
    with open(filename, "w") as file: 
        for item in my_list:
            file.write(item + '\n') 
    
def calc_zeros(alignment): 
    count = 0 
    return [0 if base == "-" else (count := count + 1) for base in alignment[0]]

def crop_alignment(start, end, alignment):
    
    zeros_seq = calc_zeros(alignment)
    try:
        ix_start = zeros_seq.index(start)
        ix_end = zeros_seq.index(end)
        cropped_alignment = alignment[:, ix_start:ix_end]
        return cropped_alignment
    except: 
        return None
    

#this is the function that's being used
def concat_alignment(gtf_file, alignment):
   # print(alignment)
    complete_align_type = MultipleSeqAlignment([]) 
    #add the groups to complete align: 
    for recordix in range(len(alignment)): 
        new_record = SeqRecord.SeqRecord("") 
        #new_record = SeqRecord("")
        new_record.id = alignment[recordix].id 
        new_record.seq = Seq("") #added this 

        complete_align_type.append(new_record)
    
   # print(complete_align_type)
    for index, row in gtf_file.iterrows():     
        crop_align = crop_alignment(row[3], row[4], alignment)
        #print(crop_align)
        if crop_align is not None: 
            for recordix in range(len(crop_align)): 
                complete_align_type[recordix].seq += crop_align[recordix].seq #append additional sequence
        
    return complete_align_type

In [29]:
#make directory for each gene pair 
genes = ["EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X"] 

gene_pair_dict = {"EIF1AX_EIF1AY": ["EIF1AX", "EIF1AY"], "KDM5C_KDM5D":["KDM5C", "KDM5D"], "KDM6A_UTY": ["KDM6A", "UTY"], "ZFX_ZFY": ["ZFX", "ZFY"], "DDX3X_DDX3Y": ["DDX3X", "DDX3Y"],
                 "USP9X_USP9Y": ["USP9X", "USP9Y"], "RPS4X_RPS4Y1": ['RPS4X', 'RPS4Y1']}

for gene_pair,two_genes in gene_pair_dict.items(): 
    subprocess.run(['mkdir', gene_pair],  stderr=subprocess.PIPE) 
    records_X = list(SeqIO.parse('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/' + two_genes[0] + ".fa", "fasta")) 
    SeqIO.write(records_X[0], '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/' + gene_pair + '/human_Xgene', 'fasta') 
    SeqIO.write(records_X[1], '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/' + gene_pair + '/chimp_Xgene', 'fasta') 

    records_Y = list(SeqIO.parse('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/' + two_genes[1]  + ".fa", "fasta")) 
    SeqIO.write(records_Y[0], '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/' + gene_pair + '/human_Ygene' , 'fasta') 
    SeqIO.write(records_Y[1], '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/' + gene_pair + '/chimp_Ygene', 'fasta') 


In [256]:
#run alignments 


for gene_pair, two_genes in gene_pair_dict.items(): 

    tree = '((human_Xgene chimp_Xgene) human_Ygene chimp_Ygene)'
    
    os.chdir('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/'+ gene_pair)

    subprocess.run(["touch", "all_bz.log"])  #making a new file 
    
    with open("all_bz.log", "w") as log_file:
        result = subprocess.run(['all_bz', '-', tree], stdout=log_file, stderr=subprocess.PIPE, check=True)
        
    make_sbatch_file("testj1.sh") #make sbatch file - it makes a new one each time through so you dont have to delete the old one 
    print(gene_pair) 
    with open("testj1.sh", "a") as file: #append to the file 
        file.write("bash ") 
        file.write("all_bz.log" + "\n")
        file.write("tba '" + tree + "' *.*.maf tba.maf >&tba.log" + "\n") 
        file.write("maf_project tba.maf human_Xgene '" + tree + "' > human_proj.maf" + "\n") #project from the perspective of the X chr 
        file.write("msa_view -o FASTA human_proj.maf > " + gene_pair + "_msa.fa")
        
    subprocess.run(["sbatch", 'testj1.sh'], stderr=subprocess.PIPE) 
    print(gene_pair)
    os.chdir('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/')  
    

KDM5C_KDM5D
Submitted batch job 712003
KDM5C_KDM5D


In [257]:
#write as phylip file
for gene_pair, two_genes in gene_pair_dict.items():     
    records = SeqIO.parse("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/" +  gene_pair + "/" +  gene_pair +  "_msa.fa", "fasta") 
    new_name = "/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/" +  gene_pair + "/" +  gene_pair + "_msa.phy"
    SeqIO.write(records, new_name, 'phylip') 
    

KDM5C_KDM5D
<generator object FastaIterator at 0x7f7981dc8f90>


In [265]:
#extract regions
gene_pair_dict = {"EIF1AX_EIF1AY": ["EIF1AX", "EIF1AY"], "KDM5C_KDM5D":["KDM5C", "KDM5D"], "KDM6A_UTY": ["KDM6A", "UTY"], "ZFX_ZFY": ["ZFX", "ZFY"], "DDX3X_DDX3Y": ["DDX3X", "DDX3Y"],
                 "USP9X_USP9Y": ["USP9X", "USP9Y"], "RPS4X_RPS4Y1": ['RPS4X', 'RPS4Y1']}

regions = ["promoter", "exon", "intron"]   

for gene_pair, two_genes in gene_pair_dict.items(): 
    alignment = AlignIO.read("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/" + gene_pair + "/" + gene_pair + "_msa.fa", "fasta")
    
    gtf_file = pd.read_csv("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/" + two_genes[0] + "_gtf_all103023.txt", delimiter="\t", header = None) 
    
    for region in regions: 
        
        gtf_file1 = gtf_file[gtf_file[2] == region] 
        #print(gtf_file1)
        returned_align = concat_alignment(gtf_file1,alignment) 
        #print(gene_pair, returned_align, region)
        SeqIO.write(returned_align, "/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/" + gene_pair + "/" + gene_pair + "_" + region + "_msa.fa", "fasta") #as fasta
        SeqIO.write(returned_align, "/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/" + gene_pair + "/" + gene_pair + "_" + region + "_msa.phy", "phylip") #as phylip


EIF1AX_EIF1AY
KDM5C_KDM5D
KDM6A_UTY
ZFX_ZFY
DDX3X_DDX3Y
USP9X_USP9Y
RPS4X_RPS4Y1


After generating the homolog alignments: 
    
    -use trimal to remove all gaps -- 
    -find the promoter %id 
    -find the exon %id 
    -find the intronic %id 

-leave the gaps in these alignments - does it make sense to mask like i did for the human sequence and then align ? 
-to start - not going to mask the alignments 


In [284]:
def edit_alignment(records_X, pair, region):

    new_alignment = MultipleSeqAlignment([])
    records_x_lost = [0,2] #take edit the alignment 
    
    for ix in records_x_lost:
        
        new_record = records_X[ix]
     
        new_alignment.append(new_record)
    
  
    SeqIO.write(new_alignment, "/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/" + pair + "/" + "human_XY_" + region + "_msa.phy", "phylip") #as phylip
    
    #use trimal to get rid of regions with gaps in X and Y
    subprocess.run(['/lab/page_scratch/hannah/trimal/source/trimal', '-in', "/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/" + pair + "/" + "human_XY_" + region + "_msa.phy", '-out', '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/' + pair + '/' + 'human_XY' + '_' + region + '_msa_filtered.phy', '-noallgaps'],  stderr=subprocess.PIPE)

    
    
    records_X = list(SeqIO.parse('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/' + pair + '/' + 'human_XY' + '_' + region + '_msa_filtered.phy', "phylip"))
    
    
    new_alignment = MultipleSeqAlignment([])
    records_x_lost = [0,1] #take edit the alignment since now its only 2 sequences long
    
    for ix in records_x_lost:
    # Replace '*' with '-' in the sequence so can remove them with trimal 
        
        modified_seq  = str(records_X[ix].seq).replace('-', 'K')
        modified_seq = str(modified_seq).replace('*', '-')
        modified_seq = str(modified_seq).replace('K', '*') #replacing the - w stars - want to keep the gaps for the calculations of % alignments 
        new_record = records_X[ix]
        new_record.seq = Seq(modified_seq)     
        new_alignment.append(new_record)
                

    SeqIO.write(new_alignment, "/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/" + pair + "/" + "human_XY_" + region + "_msa.phy", "phylip") #as phylip

    subprocess.run(['/lab/page_scratch/hannah/trimal/source/trimal', '-in', '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/' + pair + '/' + 'human_XY' + '_' + region + '_msa.phy', '-out', '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/' + pair + '/' + 'human_XY' + '_' + region + '_msa_filtered.phy', '-gt', '1'],  stderr=subprocess.PIPE)
        
    try:
        new_alignment = list(SeqIO.parse('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/' + pair + '/' + 'human_XY' + '_' + region + '_msa_filtered.phy', "phylip"))    
        return new_alignment
          
    except: 
        return []
    

## calculate percent ID

In [288]:
regions = ["exon", "intron"]  
pairs = [ "EIF1AX_EIF1AY","KDM5C_KDM5D", "KDM6A_UTY", "ZFX_ZFY", "DDX3X_DDX3Y", "USP9X_USP9Y", "RPS4X_RPS4Y1"]

gene_pair_dict = {"EIF1AX_EIF1AY": ["EIF1AX", "EIF1AY"], "KDM5C_KDM5D":["KDM5C", "KDM5D"], "KDM6A_UTY": ["KDM6A", "UTY"], "ZFX_ZFY": ["ZFX", "ZFY"], "DDX3X_DDX3Y": ["DDX3X", "DDX3Y"],
                 "USP9X_USP9Y": ["USP9X", "USP9Y"], "RPS4X_RPS4Y1": ['RPS4X', 'RPS4Y1']}

regions = ["intron", "exon"]


p = []
r = []
perc = []
combos = set()
combo_dict = {}

for gp_d in gene_pair_dict: 
    pair = gp_d
    #print(pair, region)
    
    for region in regions: 
              
            records_X = list(SeqIO.parse('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/' + pair + "/" + pair + "_" + region + "_msa.fa", "fasta")) 
          
            new_alignment = edit_alignment(records_X, pair, region)
           
            if new_alignment:
                x_gene = new_alignment[0] 
                y_gene = new_alignment[1] 
              # print(x_gene, y_gene)
                num_matches = sum(a == b for a, b in zip(x_gene, y_gene))
              # print(num_matches)
              # print(len(x_gene))
                percent_identity = (num_matches / len(x_gene)) * 100
                p.append(pair)
                r.append(region) 
                perc.append(percent_identity)
            else: 
                print(pair, region, "NONE")
                
for pair1, genes1 in gene_pair_dict.items():
    for pair2, genes2 in gene_pair_dict.items():
        combos.add((genes1[0], genes2[1]))
        combo_dict[(genes1[0], genes2[1])] = genes1[0] + "_" + genes2[1]


SingleLetterAlphabet() alignment with 2 rows and 13054 columns
CCTTCCCGCCCTGCCTGCCCGCGG--CTGGGCGCCCCACACGCG...TTC human_Xgene
--------------CTAACGGCGGTACTGGCCACTGCGACCCCA...TTC human_Ygene
EIF1AX_EIF1AY intron SingleLetterAlphabet() alignment with 2 rows and 12815 columns
CCTTCCCGCCCTGCCTGCCCGCGG**CTGGGCGCCCCACACGCG...TTC human_Xgen
**************CTAACGGCGGTACTGGCCACTGCGACCCCA...TTC human_Ygen IN LOOP
SingleLetterAlphabet() alignment with 2 rows and 432 columns
CATGCCCAAGAATAAAGGTAAAGGAGGTAAAAACAGACGCAGGG...GAT human_Xgene
CATGCCCAAGAATAAAGGTAAAGGAGGTAAAAACAGGCGCAGGG...GAT human_Ygene
EIF1AX_EIF1AY exon SingleLetterAlphabet() alignment with 2 rows and 432 columns
CATGCCCAAGAATAAAGGTAAAGGAGGTAAAAACAGACGCAGGG...GAT human_Xgen
CATGCCCAAGAATAAAGGTAAAGGAGGTAAAAACAGGCGCAGGG...GAT human_Ygen IN LOOP
SingleLetterAlphabet() alignment with 2 rows and 22411 columns
AGGGGGTGGCTGGGACCGTTGTGGTTGCGGCGCGGATCGGAAAA...CTC human_Xgene
********************************************...CTG human_Ygene
KDM5C_K

In [289]:
df = pd.DataFrame((list(zip(p,r,perc))), columns = ['pair', 'region', 'percent']) 
print(df)

             pair  region    percent
0   EIF1AX_EIF1AY  intron  52.576177
1   EIF1AX_EIF1AY    exon  92.361111
2     KDM5C_KDM5D  intron  54.673313
3     KDM5C_KDM5D    exon  83.141517
4       KDM6A_UTY  intron  57.151267
5       KDM6A_UTY    exon  87.936437
6         ZFX_ZFY  intron  57.937063
7         ZFX_ZFY    exon  92.523751
8     DDX3X_DDX3Y  intron  58.947204
9     DDX3X_DDX3Y    exon  87.867299
10    USP9X_USP9Y  intron  53.980118
11    USP9X_USP9Y    exon  85.120350
12   RPS4X_RPS4Y1  intron   0.000000
13   RPS4X_RPS4Y1    exon  82.315522


In [290]:
for combo in combos: 

    promoter_percent = get_perc(combo,  'promoter')[0] 
  
    p.append(combo_dict[combo])
    r.append("promoter")
    perc.append(promoter_percent)
    
df = pd.DataFrame((list(zip(p,r,perc))), columns = ['pair', 'region', 'percent']) 

df.to_csv('percent_alignment_1030.csv')

#extract X promoter 
#extract Y promoter 
#get combos of promoters 
#align them 

## get GC content

In [291]:
gene_pair_dict = {"EIF1AX_EIF1AY": ["EIF1AX", "EIF1AY"], "KDM5C_KDM5D":["KDM5C", "KDM5D"], "KDM6A_UTY": ["KDM6A", "UTY"], "ZFX_ZFY": ["ZFX", "ZFY"], "DDX3X_DDX3Y": ["DDX3X", "DDX3Y"],
                 "USP9X_USP9Y": ["USP9X", "USP9Y"], "RPS4X_RPS4Y1": ['RPS4X', 'RPS4Y1']}
regions = ["exon", "intron"]   

gene = []
pairs = []
reg = []
GC_perc = []

for gp_d, gl in gene_pair_dict.items(): 
    pair = gp_d
    print(pair)
    
    prom, gc_xp, gc_yp = get_perc(gl, 'promoter')
    GC_perc.append(gc_xp) 
    reg.append('promoter')
    pairs.append(pair)
    gene.append('X_gene')
    
    GC_perc.append(gc_yp)
    reg.append('promoter')
    pairs.append(pair)
    gene.append('Y_gene')
    
    for region in regions:   
        
        alignment_x = AlignIO.read("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/" + pair + "/" + "human_Xgene", "fasta")

        gtf_file_x = pd.read_csv("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/" + gl[0] + "_gtf_all103023.txt", delimiter="\t", header = None) 
        gtf_file_x = gtf_file_x[gtf_file_x[2] == region] 
        returned_align_x = concat_alignment(gtf_file_x,alignment_x) 
            
            
        alignment_y = AlignIO.read("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/" + pair + "/" + "human_Ygene", "fasta")

        gtf_file_y = pd.read_csv("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/" + gl[1] + "_gtf_all103023.txt", delimiter="\t", header = None) 
        gtf_file_y = gtf_file_y[gtf_file_y[2] == region] 
        returned_align_y = concat_alignment(gtf_file_y,alignment_y) 
    
        GC_perc.append(GC(returned_align_x[0].seq))
        reg.append(region)
        pairs.append(pair)
        gene.append('X_gene')
        
        GC_perc.append(GC(returned_align_y[0].seq))
        reg.append(region)
        pairs.append(pair)
        gene.append('Y_gene')
        
        

EIF1AX_EIF1AY
['EIF1AX', 'EIF1AY'] exon
['EIF1AX', 'EIF1AY'] intron
KDM5C_KDM5D
['KDM5C', 'KDM5D'] exon
['KDM5C', 'KDM5D'] intron
KDM6A_UTY
['KDM6A', 'UTY'] exon
['KDM6A', 'UTY'] intron
ZFX_ZFY
['ZFX', 'ZFY'] exon
['ZFX', 'ZFY'] intron
DDX3X_DDX3Y
['DDX3X', 'DDX3Y'] exon
['DDX3X', 'DDX3Y'] intron
USP9X_USP9Y
['USP9X', 'USP9Y'] exon
['USP9X', 'USP9Y'] intron
RPS4X_RPS4Y1
['RPS4X', 'RPS4Y1'] exon
['RPS4X', 'RPS4Y1'] intron


In [292]:
df1 = pd.DataFrame((list(zip(gene,pairs,reg, GC_perc))), columns = ['gene', 'pairs', 'region', 'GC_perc']) 
df1.to_csv('GC_1030.csv')

## calculate CpG for promoters 

In [373]:
def get_perc_fornormCG(combo,region): 

    gtf_file_x = pd.read_csv("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/" + combo[0] + "_gtf_all103023.txt", delimiter="\t", header = None) 
    gtf_file_x1 = gtf_file_x[gtf_file_x[2] == region] 

    gtf_file_y = pd.read_csv("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/" + combo[1] + "_gtf_all103023.txt", delimiter="\t", header = None) 
    gtf_file_y1 = gtf_file_y[gtf_file_y[2] == region] 
             
    rec_x = list(SeqIO.parse('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/' + combo[0] + "/human", "fasta"))
    rec_y = list(SeqIO.parse('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/' + combo[1] + "/human", "fasta"))

    promoter_x = rec_x[0].seq[gtf_file_x1.iloc[0,3]: gtf_file_x1.iloc[0,4]]
    #print(promoter_x)
    promoter_y = rec_y[0].seq[gtf_file_y1.iloc[0,3]: gtf_file_y1.iloc[0,4]]
    return promoter_x, promoter_y

In [382]:
gene = []
pairs = []
reg = []
CpG_norm = []
raw_cpGs = []

gene_pair_dict = {"EIF1AX_EIF1AY": ["EIF1AX", "EIF1AY"], "KDM5C_KDM5D":["KDM5C", "KDM5D"], "KDM6A_UTY": ["KDM6A", "UTY"], "ZFX_ZFY": ["ZFX", "ZFY"], "DDX3X_DDX3Y": ["DDX3X", "DDX3Y"],
                 "USP9X_USP9Y": ["USP9X", "USP9Y"], "RPS4X_RPS4Y1": ['RPS4X', 'RPS4Y1']}

for gp_d, gl in gene_pair_dict.items(): 
    pair = gp_d
    p_x, p_y = get_perc_fornormCG(gl, "promoter")
    
    p_x_CpG = (p_x.count("CG") * 500) / (p_x.count("C") * p_x.count("G"))
    p_y_CpG = (p_y.count("CG") * 500) / (p_y.count("C") * p_y.count("G"))

    
    CpG_norm.append(p_x_CpG)
    reg.append("promoter")
    pairs.append(pair)    
    gene.append('X_gene')
    raw_cpGs.append(p_x.count("CG")/500 *100)
    
    CpG_norm.append(p_y_CpG)
    reg.append("promoter")
    pairs.append(pair)    
    gene.append('Y_gene')
    raw_cpGs.append(p_y.count("CG")/500 *100)

df1 = pd.DataFrame((list(zip(gene,pairs,reg, CpG_norm, raw_cpGs))), columns = ['gene', 'pairs', 'region', 'CpG_norm', 'raw_cpGs']) 
df1.to_csv('CpG_1127.csv')

## generate alignment for x and y promoters (defined by gtf coordinates)

In [2]:
def get_perc(combo,region): 

    gtf_file_x = pd.read_csv("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/" + combo[0] + "_gtf_all103023.txt", delimiter="\t", header = None) 
    gtf_file_x1 = gtf_file_x[gtf_file_x[2] == region] 

    gtf_file_y = pd.read_csv("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/" + combo[1] + "_gtf_all103023.txt", delimiter="\t", header = None) 
    gtf_file_y1 = gtf_file_y[gtf_file_y[2] == region] 
             
    rec_x = list(SeqIO.parse('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/' + combo[0] + "/human", "fasta"))
    rec_y = list(SeqIO.parse('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/' + combo[1] + "/human", "fasta"))

    promoter_x = rec_x[0].seq[gtf_file_x1.iloc[0,3]: gtf_file_x1.iloc[0,4]]
  
    promoter_y = rec_y[0].seq[gtf_file_y1.iloc[0,3]: gtf_file_y1.iloc[0,4]]
   
    
    GC_x = GC(promoter_x)
    GC_y = GC(promoter_y)

    alignments = pairwise2.align.globalxx(promoter_x, promoter_y) 
   
    
    best_alignment = max(alignments, key=lambda alignment: alignment[2])
   
    #Get the aligned sequences from the best alignment
    aligned_x, aligned_y, score, start, end = best_alignment
 
    #Get the aligned sequences from the best alignment  
    aligned_seq1 = aligned_x 
    aligned_seq2 = aligned_y

    num_matches = sum(a == b for a, b in zip(aligned_seq1, aligned_seq2))

    percent_identity = (num_matches / len(aligned_seq1)) * 100 

    return percent_identity, GC_x, GC_y, best_alignment


## rolling alignment of 50 bp from X perspective:

In [389]:
p = []
r = []
perc = []
end_nums = []
start_nums = []

gene_pair_dict = {"EIF1AX_EIF1AY": ["EIF1AX", "EIF1AY"], "KDM5C_KDM5D":["KDM5C", "KDM5D"], "KDM6A_UTY": ["KDM6A", "UTY"], "ZFX_ZFY": ["ZFX", "ZFY"], "DDX3X_DDX3Y": ["DDX3X", "DDX3Y"],
                 "USP9X_USP9Y": ["USP9X", "USP9Y"], "RPS4X_RPS4Y1": ['RPS4X', 'RPS4Y1']}

for gp_d, gl in gene_pair_dict.items(): 

    promoter_percent, GC_x, GC_y, best_alignment = get_perc(gl,  'promoter') #different alignment method
    
    
    for num in range(1, 450, 1): #change from 25 to 1 and 475 to 499
        end_num = num + 50
        crop_align_best = crop_alignment(num,end_num, best_alignment)

        aligned_seq1 = crop_align_best[0] 
        aligned_seq2 = crop_align_best[1]

        num_matches = sum(a == b for a, b in zip(aligned_seq1, aligned_seq2))

    
        percent_identity = (num_matches / len(aligned_seq1)) * 100 
    
        p.append(gp_d)
        r.append("promoter")
        perc.append(percent_identity)
        end_nums.append(end_num)
        start_nums.append(num)
    
    
    
    
df = pd.DataFrame((list(zip(p,r,perc, end_nums, start_nums))), columns = ['pair', 'region', 'percent', "end_nums", "start_nums"]) 


0
50
1
51
2
52
3
53
4
54
5
55
6
56
7
57
8
58
9
59
10
60
11
61
12
62
13
63
14
64
15
65
16
66
17
67
18
68
19
69
20
70
21
71
22
72
23
73
24
75
25
76
26
77
27
78
28
79
29
80
30
81
31
82
32
83
33
84
34
85
35
86
36
88
37
89
38
90
39
91
40
92
41
93
42
94
43
95
44
96
45
98
46
99
47
100
48
102
49
103
50
104
51
105
52
107
53
108
54
109
55
110
56
111
57
112
58
113
59
115
60
116
61
117
62
118
63
119
64
122
65
123
66
124
67
125
68
126
69
127
70
128
71
129
72
130
73
131
75
132
76
133
77
134
78
135
79
136
80
137
81
138
82
139
83
140
84
141
85
142
86
143
88
145
89
147
90
148
91
150
92
151
93
152
94
153
95
154
96
155
98
156
99
157
100
158
102
159
103
160
104
161
105
163
107
164
108
166
109
168
110
169
111
170
112
171
113
172
115
173
116
174
117
175
118
178
119
179
122
181
123
182
124
183
125
184
126
185
127
186
128
189
129
190
130
191
131
192
132
196
133
197
134
203
135
204
136
205
137
206
138
209
139
210
140
211
141
212
142
213
143
214
145
215
147
217
148
218
150
219
151
223
152
224
153
225
154
229
15

In [391]:
df.to_csv('perc_rolling_1208.csv')

## get GC content promoters (from X perspective) 

In [8]:
g = []
gc = []
type_ofgene = []
regions1 = []

regions = [ "promoter"]  
gene_pair_dict = {"EIF1AX_EIF1AY": ["EIF1AX", "EIF1AY"], "KDM5C_KDM5D":["KDM5C", "KDM5D"], "KDM6A_UTY": ["KDM6A", "UTY"], "ZFX_ZFY": ["ZFX", "ZFY"], "DDX3X_DDX3Y": ["DDX3X", "DDX3Y"],
                 "USP9X_USP9Y": ["USP9X", "USP9Y"], "RPS4X_RPS4Y1": ['RPS4X', 'RPS4Y1']}



for gp, gl in gene_pair_dict.items(): 
            print(gp) 
            if gp == "KDM5C_KDM5D":
                continue
            records = AlignIO.read("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/pair_align/" + gp + "/" + "human_XY_promoter_msa_filtered.phy", "phylip")
            human_seq = str(records[0].seq)
            animal_seq = str(records[1].seq)
            
            h_gc = ((human_seq.count('G') + human_seq.count('C')) / (human_seq.count('G') + human_seq.count('C') + human_seq.count('A') + human_seq.count('T'))) *100
            a_gc = ((animal_seq.count('G') + animal_seq.count('C')) / (animal_seq.count('G') + animal_seq.count('C') + animal_seq.count('A') + animal_seq.count('T'))) *100
            
            gc.append(h_gc)
            g.append(gp)
            type_ofgene.append('X_gene')
            
            gc.append(a_gc) 
            g.append(gp)
            type_ofgene.append('Y_gene')

            


EIF1AX_EIF1AY
KDM5C_KDM5D
KDM6A_UTY
ZFX_ZFY
DDX3X_DDX3Y
USP9X_USP9Y
RPS4X_RPS4Y1


In [9]:
df = pd.DataFrame((list(zip(g, gc, type_ofgene))), columns = ['pair', 'gc', 'type']) 


In [10]:
df

,pair,gc,type
0,EIF1AX_EIF1AY,61.016949,X_gene
1,EIF1AX_EIF1AY,51.898734,Y_gene
2,KDM6A_UTY,70.800000,X_gene
3,KDM6A_UTY,54.352442,Y_gene
4,ZFX_ZFY,78.800000,X_gene
5,ZFX_ZFY,70.488323,Y_gene
6,DDX3X_DDX3Y,54.600000,X_gene
7,DDX3X_DDX3Y,50.626566,Y_gene
8,USP9X_USP9Y,69.000000,X_gene
9,USP9X_USP9Y,51.302605,Y_gene
